In [ ]:
!pip3 install websockets pandas ta python-binance python-dotenv

In [16]:
import websockets
import json
import pandas as pd
import asyncio
import ta
import os
from binance.client import Client

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.



api_key = os.environ['api_key']
api_secret = os.environ['api_secret']
quantity = os.environ['quantity']
roc_num_records = int(os.environ['roc_num_records'])
symbol = os.environ['symbol']
trailing_coefficient = float(os.environ['trailing_coefficient'])
rise_percentage = float(os.environ['rise_percentage'])


#PROD
stream = websockets.connect(f"wss://stream.binance.com:9443/stream?streams={symbol.lower()}@miniTicker")

#TEST
#stream = websockets.connect('wss://testnet.binance.vision/stream?streams=adausdt@miniTicker')

In [17]:
# Functions
def createframe(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:,['s', 'E', 'c']]
  df.columns = ['symbol', 'Time', 'Price']
  df.Price = df.Price.astype('float')
  df.Time = pd.to_datetime(df.Time, unit='ms')
  return df

# Variables
client = Client(api_key, api_secret)
df = pd.DataFrame()
open_position = False
dfOrders = pd.DataFrame()


In [18]:
  # TODO check quantity >= minQty, if not sto execution
  symbolinfo = client.get_symbol_info(symbol)

In [19]:
# TODO make quantity, symbol, roc test rows umber, trailing stop coefficient dynamic
async with stream as receiver: 
  print(f"Min qty for {symbol} is {symbolinfo['filters'][2]['minQty']}")
  print(f"Starting trailing strategy for symbol {quantity} of {symbol}")
  while True:
    data = await receiver.recv()
    data = json.loads(data)['data']
    df = df.append(createframe(data))
    
    if len(df) > 30:
      # If not open position then calculate ROC with roc_num_records records
      if not open_position:
        # TODO understand second condition... place buy order, 
        # save buyprice for later use        
        if ta.momentum.roc(df.Price, roc_num_records).iloc[-1] > 0 and ta.momentum.roc(df.Price, roc_num_records).iloc[-2]:
          print(f"ROC = {ta.momentum.roc(df.Price, roc_num_records).iloc[-1]}")
          order = client.create_order(symbol=symbol, side=Client.SIDE_BUY, type=Client.ORDER_TYPE_MARKET, quantity=quantity)
          dfOrders.append(pd.DataFrame([order,'BUY'], columns=['ORDER', 'TYPE']))
          print(dfOrders)
          open_position = True
          buyprice = float(order['fills'][0]['price'])

      if open_position:
        subdf = df[df.Time >= pd.to_datetime(
            order['transactTime'], unit='ms')]
        if len(subdf) > 1:

          #Use cummax function to calculate highest price
          subdf['highest'] = subdf.Price.cummax()

          # Setting trailing stop
          # TODO make trailing coefficient variable
          subdf['trailingstop'] = subdf['highest'] * trailing_coefficient

          # If any of these 2 condition is true i want to place a SELL ORDER
          # If the asset is rising 0.02%
          if subdf.iloc[-1].Price < subdf.iloc[-1].trailingstop or df.iloc[-1].Price / float(order['fills'][0]['price']) > rise_percentage:
            order = client.create_order(symbol=symbol, side=Client.SIDE_SELL, type=Client.ORDER_TYPE_MARKET, quantity=quantity)
            dfOrders.append(pd.DataFrame([order,'SELL'], columns=['ORDER', 'TYPE']))
            print(dfOrders)

            sellprice = float(order['fills'][0]['price'])
            print(f"You made {(sellprice - buyprice)/2} profit")
            open_position = False

    #print(df.iloc[-1])


Min qty for ETHUSDT is 0.00010000
Starting trailing strategy for symbol 0.001 of ETHUSDT
ROC = 0.00045437609617158745


BinanceAPIException: APIError(code=-1013): Filter failure: MIN_NOTIONAL